In [1]:
# importing the required packages 
import cv2 
from zipfile import ZipFile
import shutil
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Download rclone from the website - Youtube link: https://www.youtube.com/watch?v=U6YPgARhRzA
!wget https://downloads.rclone.org/v1.53.2/rclone-v1.53.2-linux-amd64.deb
!apt install ./rclone-v1.53.2-linux-amd64.deb

--2020-11-24 00:10:07--  https://downloads.rclone.org/v1.53.2/rclone-v1.53.2-linux-amd64.deb
Resolving downloads.rclone.org (downloads.rclone.org)... 5.153.250.7, 2a02:24e0:8:61f9::1
Connecting to downloads.rclone.org (downloads.rclone.org)|5.153.250.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13197872 (13M) [application/x-debian-package]
Saving to: ‘rclone-v1.53.2-linux-amd64.deb’

rclone-v1.53.2-linu 100%[===================>]  12.59M  8.87MB/s    in 1.4s    

2020-11-24 00:10:10 (8.87 MB/s) - ‘rclone-v1.53.2-linux-amd64.deb’ saved [13197872/13197872]

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'rclone' instead of './rclone-v1.53.2-linux-amd64.deb'
The following NEW packages will be installed:
  rclone
0 upgraded, 1 newly installed, 0 to remove and 14 not upgraded.
Need to get 0 B/13.2 MB of archives.
After this operation, 41.1 MB of additional disk space will be used.
Get:1 /content/

In [3]:
!rclone config

2020/11/24 00:10:16 NOTICE: Config file "/root/.config/rclone/rclone.conf" not found - using defaults
No remotes found - make a new one
n) New remote
s) Set configuration password
q) Quit config
n/s/q> n
name> onedrive
Type of storage to configure.
Enter a string value. Press Enter for the default ("").
Choose a number from below, or type in your own value
 1 / 1Fichier
   \ "fichier"
 2 / Alias for an existing remote
   \ "alias"
 3 / Amazon Drive
   \ "amazon cloud drive"
 4 / Amazon S3 Compliant Storage Provider (AWS, Alibaba, Ceph, Digital Ocean, Dreamhost, IBM COS, Minio, Tencent COS, etc)
   \ "s3"
 5 / Backblaze B2
   \ "b2"
 6 / Box
   \ "box"
 7 / Cache a remote
   \ "cache"
 8 / Citrix Sharefile
   \ "sharefile"
 9 / Dropbox
   \ "dropbox"
10 / Encrypt/Decrypt a remote
   \ "crypt"
11 / FTP Connection
   \ "ftp"
12 / Google Cloud Storage (this is not Google Drive)
   \ "google cloud storage"
13 / Google Drive
   \ "drive"
14 / Google Photos
   \ "google photos"
15 / Hubic
   

In [4]:
# Mount OneDrive for dataset
!sudo mkdir /content/onedrive/
!nohup rclone --vfs-cache-mode writes mount onedrive: /content/onedrive/ &

nohup: appending output to 'nohup.out'


In [ ]:
ls

nohup.out  onedrive/  rclone-v1.53.2-linux-amd64.deb  sample_data/


In [5]:
cd onedrive/First_Impression_V2_Dataset/

/content/onedrive/First_Impression_V2_Dataset


In [ ]:
# unzip folders containing the data from the drive the proceed

In [6]:
# ************ Extracting training videos ************  
# l is list of  the folder numbers inside the training data folders 
f1 = 1 # lowest folder number 
f2 = 13 # highest folder number + 1
l = list(map(str,range(f1,f2)))
for i in l:
  file_name = 'training80_'+i+'.zip'
  folder_to_extract = 'training80_'+i
  print(folder_to_extract)
  # make directory of the same name 
  os.mkdir(folder_to_extract)

  zf = ZipFile(file_name, 'r')
  zf.extractall(folder_to_extract)
  zf.close()

  # delete orginal folder
  os.remove(file_name)

training80_1


FileExistsError: ignored

In [ ]:
count = 0
# getting the total number of videos in the training data folder  
for i in l:
  folder_name = 'training80_'+i # Change here for validation, test sets
  count += len(os.listdir(folder_name))
print('total training video files are: ',count)

total training video files are:  960


In [7]:
# ************ Extracting validation videos ************ 
# l is list of  the folder numbers inside the validation data folders 
f1 = 1 # lowest folder number 
f2 = 13 # highest folder number + 1
l_1 = list(map(str,range(f1,f2)))
for i in l_1:
  file_name = 'validation80_'+i+'.zip'
  folder_to_extract = 'validation80_'+i
  print(folder_to_extract)
  # make directory of the same name 
  os.mkdir(folder_to_extract)

  zf = ZipFile(file_name, 'r')
  zf.extractall(folder_to_extract)
  zf.close()

  # delete orginal folder
  os.remove(file_name)


validation80_1


FileExistsError: ignored

In [ ]:
count = 0
# getting the total number of videos in the validation data folder  
for i in l:
  folder_name = 'test80_'+i # Change here for validation, test sets
  count += len(os.listdir(folder_name))
print('total validation video files are: ',count)

total validation video files are:  960


In [9]:
# lets look at the avg length of the videos 

def count_frames(path):
  video = cv2.VideoCapture(path)
  total = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
  rate = video.get(cv2.CAP_PROP_FPS)
  return total, rate

In [ ]:
for i in l:
  total_time = 0
  folder_name = 'validation80_'+i # Change here for validation, test sets
  for j in os.listdir(folder_name):
    path = folder_name+'/'+j
    total, rate = count_frames(path)
    time = total / rate 
    total_time += time
  print(f'Avg length of videos in folder {folder_name} is {total_time/len(os.listdir(folder_name))} secs')

In [ ]:
# Check if videos are in RGB format 
# cv2.CAP_PROP_CONVERT_RGB is a booloean flag indicating if the videos need to be converted to RGB or not i.e. False/ 0 = no need to convert 

# Loop through all video files and see if any video needs to be converted i.e. its not in RGB format 
for i in l:
  total_time = 0
  folder_name = 'test80_'+i # Change here for validation, test sets
  print('Entering folder ',folder_name)
  for j in os.listdir(folder_name):
    path = folder_name+'/'+j
    video = cv2.VideoCapture(path)
    rgb_flg = video.get(cv2.CAP_PROP_CONVERT_RGB)

    if rgb_flg == True:
      print(path, 'Needs to be converted') 
      
# All flags were false hence no need to convert 

Entering folder  test80_1
Entering folder  test80_2
Entering folder  test80_3
Entering folder  test80_4
Entering folder  test80_5
Entering folder  test80_6
Entering folder  test80_7
Entering folder  test80_8
Entering folder  test80_9
Entering folder  test80_10
Entering folder  test80_11
Entering folder  test80_12


In [10]:
# function to extract 15 frames from each video and store it in a folder 

def extract_frames(path, flag = 'train'):

  name = path.split('/')[1]
  file_name = name.split('.mp4')[0]
  cap = cv2.VideoCapture(path)

  # make folder to save extracted frames 
  try:
    if flag == 'validation':
      path_ = 'val_frames/'
      os.makedirs(path_ + file_name)
    elif flag == 'test':
      path_ = 'test_frames/'
      os.makedirs(path_ + file_name)
    else:
      path_ = 'frames/'
      os.makedirs(path_ + file_name)
  except FileExistsError:
    print('Folder already exists')

  # counter variable 
  count =0 
  # flag for knowing if frames were successfully extracted 
  success = 1

  total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

  # set frame limit to 100 using cap.set() which sets the camera parameters 
  cap.set(cv2.CAP_PROP_FRAME_COUNT, 16) # starts count from 1 not 0 hence, 16 not 15


  while success:
    count +=1

    if count >= total_frames or count == 16:
      break

    success, frame = cap.read()
    # resizing frame to save space  and also for modelling
    frame = cv2.resize(frame,(150, 150), interpolation= cv2.INTER_AREA)  # cv2.INTER_AREA good for shrinking images 
    # interpolation is used to estimate the values of the unknown pixels, bicubic interpolation uses 4X4  pixels 
    # i.e. it uses weighed average of 16 pixels, unknown pixels can be at different distances. it gives higher weight to closer pixels. 
    
    # save the frame
    frame_path = path_ + file_name + '/' + file_name +'_' +str(count) + '.jpg'
    
    cv2.imwrite(frame_path, frame)

    if cv2.waitKey(1) & 0xFF == ord('q'): # give the user one millisecond to press 'q' key to abort
      break
  
  cap.release()

In [10]:
# ************* Extracting frames from Train Videos ******************
try:
  os.makedirs('frames') # This folder will contain the folders for each video which will contain 15 frames for that video 
except FileExistsError:
    print('Frames Folder already exists')

# loop to extract the frames 
for i in l:
  folder_name = 'training80_'+i
  print('Entering folder ',folder_name)

  for j in os.listdir(folder_name):
    path = folder_name+'/'+j
    extract_frames(path)

Frames Folder already exists


NameError: ignored

In [ ]:
# ************* Extracting frames from Validation Videos ******************
try:
  os.makedirs('val_frames') # This folder will contain the folders for each video which will contain 15 frames for that video 
except FileExistsError:
    print('Validation Frames Folder already exists')

# loop to extract the frames 
for i in l:
  folder_name = 'validation80_'+i
  print('Entering folder ',folder_name)

  for j in os.listdir(folder_name):
    path = folder_name+'/'+j
    extract_frames(path, flag = 'validation')

Validation Frames Folder already exists
Entering folder  validation80_1
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Fold

In [14]:
i

'7'

In [11]:
# ************* Extracting frames from Test Videos ******************
try:
  os.makedirs('test_frames') # This folder will contain the folders for each video which will contain 15 frames for that video 
except FileExistsError:
    print('Test Frames Folder already exists')

# loop to extract the frames 
for i in l:
  folder_name = 'test80_'+i
  print('Entering folder ',folder_name)

  #count = 0
  for j in os.listdir(folder_name):
    path = folder_name+'/'+j
    extract_frames(path, flag = 'test')

      #count +=1
      #if count %10 ==0:
      #print(count)

Test Frames Folder already exists
Entering folder  test80_1
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already exists
Folder already e

In [ ]:
!zip -r "/content/onedrive/First_Impression_V2_Dataset/frames_1.zip" "/content/onedrive/First_Impression_V2_Dataset/frames"  # zip the folder containing the frames and store it on the drive

In [ ]:
zip -r /content/onedrive/First_Impression_V2_Dataset/frames_1.zip . -i /content/onedrive/First_Impression_V2_Dataset/frames

In [ ]:
!zip -r onedrive/'First_Impression_V2_Dataset'/frames_1.zip frames # zip the folder containing the frames and store it on the drive

In [ ]:
!zip -r onedrive/'First_Impression_V2_Dataset'/val_frames_1.zip val_frames # zip the folder containing the frames and store it on the drive

In [ ]:
!zip -r onedrive/'First_Impression_V2_Dataset'/test_1.zip test_frames # zip the folder containing the frames and store it on the drive